# Tutorial: Red Neuronal Multicapa (Backpropagation) — implementación from‑scratch para XOR

Notebook didáctico que cubre fundamentos teóricos, matemática del forward/backward y una implementación completa *from‑scratch* en Python (numpy) para aprender la compuerta XOR.

## 1. Fundamentos teóricos

- Una Red Neuronal Multicapa (MLP) consta de capas: entrada → capa oculta(s) → salida.
- Para problemas no linealmente separables (como XOR) se necesita al menos una capa oculta con activación no lineal.
- Backpropagation: calcular gradientes usando la regla de la cadena y actualizar pesos con un optimizador (p. ej. gradiente descendente).

## 2. Fundamento matemático

### 2.1 Forward Pass (notación vectorizada)

- X: (batch, n_in). Pesos: W1 (n_in, n_h), b1 (1, n_h), W2 (n_h, n_out), b2 (1, n_out).
- Z1 = X·W1 + b1; A1 = σ(Z1).
- Z2 = A1·W2 + b2; A2 = σ(Z2) (salida probabilística).

### 2.2 Backward Pass (gradientes)

- Error: E = Y - A2.
- delta2 = E * σ'(A2)  (σ'(a)=a*(1-a) si a = σ(z)).
- gradW2 = A1^T · delta2; gradb2 = sum(delta2, axis=0).
- delta1 = delta2 · W2^T * σ'(A1).
- gradW1 = X^T · delta1; gradb1 = sum(delta1, axis=0).
- Actualizar: W += lr * gradW  (si error = Y - Yhat y usamos signo positivo en actualización como en ejemplos didácticos).

## 3. Implementación from‑scratch

A continuación definimos la clase SimpleMLP con una capa oculta y entrenamiento por batch usando backpropagation.

## 4. Librerías, clases y funciones

- numpy: operaciones vectorizadas y manejo de matrices.
- sklearn.metrics: accuracy_score, confusion_matrix (para evaluación final).
- Funciones definidas en código: sigmoid, sigmoid_derivative, forward, backward, train, predict_proba, predict, check_all_inputs.

## 5. Pipeline

### Model Selection

Para resolver el problema XOR se seleccionó una Red Neuronal Multicapa (MLP) con una arquitectura mínima de 2–2–1 (dos neuronas de entrada, dos en la capa oculta y una de salida). Esta configuración es la más sencilla capaz de representar funciones no lineales, ya que el problema XOR no es linealmente separable, lo que impide resolverlo mediante un perceptrón simple o modelos lineales.

El uso de una función de activación sigmoide en la capa oculta y en la salida permite introducir no linealidad en el modelo, lo cual es indispensable para que la red pueda aprender relaciones complejas entre las entradas y las salidas. Además, la sigmoide proporciona salidas continuas en el rango (0, 1), adecuadas para tareas de clasificación binaria como esta.

La decisión de entrenar la red “from scratch” obedece a fines de transparencia y comprensión del algoritmo, permitiendo visualizar cada etapa del proceso de aprendizaje: propagación hacia adelante, cálculo del error, retropropagación de gradientes y actualización de pesos. De esta forma se obtiene una comprensión más profunda del funcionamiento interno de la retropropagación antes de usar librerías de alto nivel como scikit-learn.

### Model Training

In [1]:
# Implementación from-scratch (numpy)
import numpy as np
from typing import Tuple

class SimpleMLP:
    def __init__(self, n_inputs: int, n_hidden: int, n_outputs: int, lr: float = 0.5, seed: int = 1):
        rng = np.random.RandomState(seed)
        # Inicialización uniforme pequeña
        self.W1 = rng.uniform(-1.0, 1.0, (n_inputs, n_hidden))
        self.b1 = np.zeros((1, n_hidden))
        self.W2 = rng.uniform(-1.0, 1.0, (n_hidden, n_outputs))
        self.b2 = np.zeros((1, n_outputs))
        self.lr = lr
    
    def sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1.0 / (1.0 + np.exp(-x))
    
    def sigmoid_deriv(self, y: np.ndarray) -> np.ndarray:
        # y = sigmoid(x)
        return y * (1 - y)
    
    def forward(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        Z1 = X.dot(self.W1) + self.b1            # (batch, n_hidden)
        A1 = self.sigmoid(Z1)                    # (batch, n_hidden)
        Z2 = A1.dot(self.W2) + self.b2           # (batch, n_outputs)
        A2 = self.sigmoid(Z2)                    # (batch, n_outputs)
        return Z1, A1, Z2, A2
    
    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 10000, print_every: int = 1000):
        # X: (N, n_inputs), y: (N, n_outputs)
        for epoch in range(1, epochs + 1):
            Z1, A1, Z2, A2 = self.forward(X)
            error = y - A2                         # (N, n_outputs)
            loss = np.mean(np.square(error))

            # Backpropagation (vectorizado)
            delta2 = error * self.sigmoid_deriv(A2)            # (N, n_outputs)
            gradW2 = A1.T.dot(delta2)                          # (n_hidden, n_outputs)
            gradb2 = np.sum(delta2, axis=0, keepdims=True)     # (1, n_outputs)

            delta1 = delta2.dot(self.W2.T) * self.sigmoid_deriv(A1)  # (N, n_hidden)
            gradW1 = X.T.dot(delta1)                                 # (n_inputs, n_hidden)
            gradb1 = np.sum(delta1, axis=0, keepdims=True)           # (1, n_hidden)

            # Actualización de pesos (nota: error = y - y_hat, por eso sumamos lr*grad)
            self.W2 += self.lr * gradW2
            self.b2 += self.lr * gradb2
            self.W1 += self.lr * gradW1
            self.b1 += self.lr * gradb1

            if (epoch % print_every) == 0:
                print(f"Epoch {epoch}, loss={loss:.6f}")

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        _, _, _, A2 = self.forward(X)
        return A2
    
    def predict(self, X: np.ndarray, threshold: float = 0.5) -> np.ndarray:
        probs = self.predict_proba(X)
        return (probs >= threshold).astype(int)

In [2]:
# Preparar datos (tabla de verdad XOR)
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

# Instanciar y entrenar
mlp = SimpleMLP(n_inputs=2, n_hidden=2, n_outputs=1, lr=0.8, seed=2)
mlp.train(X, y, epochs=8000, print_every=2000)

Epoch 2000, loss=0.001533
Epoch 4000, loss=0.000545
Epoch 6000, loss=0.000326
Epoch 8000, loss=0.000231


### Prediction

In [3]:
# Predicción y comprobación por entrada
def check_all_inputs_fromscratch(model: SimpleMLP, X: np.ndarray):
    probs = model.predict_proba(X)
    preds = model.predict(X)
    for xi, pi, pr in zip(X, preds, probs):
        print(f"entrada={xi}, prob={pr.ravel()[0]:.4f}, pred={pi.ravel()[0]}")

print('\nComprobación detallada (from-scratch):')
check_all_inputs_fromscratch(mlp, X)

print('\nPredicciones (vector):', mlp.predict(X).flatten())


Comprobación detallada (from-scratch):
entrada=[0 0], prob=0.0137, pred=0
entrada=[0 1], prob=0.9855, pred=1
entrada=[1 0], prob=0.9855, pred=1
entrada=[1 1], prob=0.0178, pred=0

Predicciones (vector): [0 1 1 0]


### Model Evaluation

- Las probabilidades cercanas a 0.5 pueden derivar en decisiones sensibles al umbral. Ajustar semilla/inicialización, lr o epochs puede estabilizar el mapeo exacto a la tabla XOR.

In [4]:
# Evaluación: Accuracy y Confusion Matrix
from sklearn.metrics import accuracy_score, confusion_matrix

y_true = y.flatten()
y_pred = mlp.predict(X).flatten()
acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
print(f"Accuracy (from-scratch): {acc:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy (from-scratch): 1.0000
Confusion Matrix:
 [[2 0]
 [0 2]]


## 6. Explicación breve de parámetros y funciones clave (python)

- lr (learning rate): controla el tamaño del paso en la actualización de pesos.
- epochs: número de iteraciones completas sobre el conjunto de entrenamiento.
- seed: semilla para inicializar aleatoriamente los pesos para reproducibilidad.
- sigmoid / sigmoid_deriv: activación logística y su derivada, utilizadas en forward y backprop.
- predict_proba / predict: devuelven probabilidades de clase y etiquetas binarias con umbral.